In [3]:
import random, json

class Sentiment:
    NEGATIVE = "NEGATIVE"
    NEUTRAL = "NEUTRAL"
    POSITIVE = "POSITIVE"

class Review:
    def __init__(self, text, score):
        self.text = text
        self.score = score
        self.sentiment = self.get_sentiment()
        
    def get_sentiment(self):
        if self.score <= 2:
            return Sentiment.NEGATIVE
        elif self.score == 3:
            return Sentiment.NEUTRAL
        else: #score of 4 or 5
            return Sentiment.POSITIVE
        
class ReviewContainer:
    def __init__(self, reviews):
        self.reviews = reviews
        
    def get_text(self):
        return [x.text for x in self.reviews]
    
    def get_sentiment(self):
        return [x.sentiment for x in self.reviews]
        
    def evenly_distribute(self):
        #creates container for x.sentiment where sentiment is NEGATIVE and POSITIVE
        negative = list(filter(lambda x: x.sentiment == Sentiment.NEGATIVE, self.reviews))
        positive = list(filter(lambda x: x.sentiment == Sentiment.POSITIVE, self.reviews))
        #makes positive = to len of negative
        positive_shrunk = positive[:len(negative)] 
        self.reviews = negative + positive_shrunk
        #if not shuffle, all pos will appear first, then neg
        random.shuffle(self.reviews)
        #print(negative[0].text)
        #print(len(negative))
        #print(len(positive))


        

In [4]:

file_name = "./data/sentiment/books_small_10000.json"

reviews = []

with open(file_name) as f:
    for line in f:
        review = json.loads(line)
        reviews.append(Review(review["reviewText"], review["overall"]))
        
reviews[5].text


'I hoped for Mia to have some peace in this book, but her story is so real and raw.  Broken World was so touching and emotional because you go from Mia\'s trauma to her trying to cope.  I love the way the story displays how there is no "just bouncing back" from being sexually assaulted.  Mia showed us how those demons come for you every day and how sometimes they best you. I was so in the moment with Broken World and hurt with Mia because she was surrounded by people but so alone and I understood her feelings.  I found myself wishing I could give her some of my courage and strength or even just to be there for her.  Thank you Lizzy for putting a great character\'s voice on a strong subject and making it so that other peoples story may be heard through Mia\'s.'

In [6]:
#training, test = train_test_split(reviews, test_size=0.33, random_state=42)

#train_container = ReviewContainer(training)
#test_container = ReviewContainer(test)

import sys
sys.path.append('./venv/lib/python3.8/site-packages')

from sklearn.model_selection import train_test_split
training, test = train_test_split(reviews, test_size=0.33, random_state=42)

train_container = ReviewContainer(training)

test_container = ReviewContainer(test)


In [7]:
train_container.evenly_distribute()
train_x = train_container.get_text()
train_y = train_container.get_sentiment()

test_container.evenly_distribute()
test_x = test_container.get_text()
test_y = test_container.get_sentiment()

train_y.count(Sentiment.POSITIVE)
train_y.count(Sentiment.NEGATIVE)

436

In [32]:
#Bag of words vectorization

In [9]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

# This book is great
# This book was so bad
# Count vectorizer (term-frequency) weighs word appearance equally
# 'This' appears in both, and weighs same as 'great'

# Term-frequency inverse-document frequency
# Gives less weight to words that appear more frequently
# Indicating there's more meaning to be inferred 
# from words that appear less often

vectorizer = TfidfVectorizer()
train_x_vectors = vectorizer.fit_transform(train_x)

test_x_vectors = vectorizer.transform(test_x)


#vectorizer = CountVectorizer()
#train_x_vectors = vectorizer.fit_transform(train_x)

#test_x_vectors = vectorizer.transform(test_x)


print(train_x[0])
print(train_x_vectors[0])

This would have to have been the worst of Lynda La Plantes books that I've read. It took ages to get into the story and I felt that there was a lot of padding. If I could have given the story zero stars I would have. It's one of those books that you'd love to have your money back for having bought it as it promises more and delivers little. A very weak story! Don't waste your money!
  (0, 8614)	0.13044101240659642
  (0, 2383)	0.09274844476658857
  (0, 8634)	0.1529832650486529
  (0, 8497)	0.07267585752376987
  (0, 4722)	0.09517294430585763
  (0, 2096)	0.19606481431716546
  (0, 5185)	0.07166513032361159
  (0, 6187)	0.20797753575303438
  (0, 569)	0.06947767307619718
  (0, 1022)	0.14342222927061166
  (0, 3671)	0.12305729746526922
  (0, 3177)	0.05073787731272773
  (0, 698)	0.10228711252975144
  (0, 5168)	0.24611459493053844
  (0, 8883)	0.18101771067661412
  (0, 4782)	0.08400677012341512
  (0, 8879)	0.06463043055276674
  (0, 7984)	0.11340572113445474
  (0, 5514)	0.06635580213104168
  (0, 745

In [10]:
from sklearn import svm
# various algos are fit to the training data(svm, dec, gnb, log)
clf_svm = svm.SVC(kernel='linear')

# here we fit the model to our training data 
clf_svm.fit(train_x_vectors, train_y)

test_x[0]
test_x_vectors[0]
# here we make a prediction on the first test sample
clf_svm.predict(test_x_vectors[0])

array(['POSITIVE'], dtype='<U8')

In [11]:
### Decision tree
from sklearn.tree import DecisionTreeClassifier

clf_dec = DecisionTreeClassifier()
clf_dec.fit(train_x_vectors, train_y)

clf_dec.predict(test_x_vectors[0])

array(['POSITIVE'], dtype='<U8')

In [12]:
### Naive Bayes
from sklearn.naive_bayes import GaussianNB

clf_gnb = GaussianNB()
clf_gnb.fit(train_x_vectors.toarray(), train_y)

clf_gnb.predict(test_x_vectors[0].toarray())

array(['NEGATIVE'], dtype='<U8')

In [51]:
### Logistic Regression

In [13]:
from sklearn.linear_model import LogisticRegression
#from sklearn import preprocessing


clf_lr = LogisticRegression(max_iter=10000)
#train_x_pre = preprocessing.scale(train_x_vectors)
clf_lr.fit(train_x_vectors, train_y)

clf_dec.predict(test_x_vectors[0])

array(['POSITIVE'], dtype='<U8')

In [14]:
# EVALUATION

In [14]:
# here we get the accuracy score of the classification 
print(clf_svm.score(test_x_vectors, test_y))
print(clf_dec.score(test_x_vectors, test_y))
print(clf_gnb.score(test_x_vectors.toarray(), test_y))
print(clf_lr.score(test_x_vectors, test_y))

0.8076923076923077
0.6466346153846154
0.6610576923076923
0.8052884615384616


In [30]:
# here we get the accuracy score of the classification 
print(clf_svm.score(test_x_vectors, test_y))
print(clf_dec.score(test_x_vectors, test_y))
print(clf_gnb.score(test_x_vectors.toarray(), test_y))
print(clf_lr.score(test_x_vectors, test_y))

0.8076923076923077
0.6466346153846154
0.6610576923076923
0.8052884615384616


In [16]:
clf_svm.score(test_x_vectors, test_y)

0.8076923076923077

In [17]:
clf_dec.score(test_x_vectors, test_y)

0.6466346153846154

In [18]:
clf_gnb.score(test_x_vectors.toarray(), test_y)

0.6610576923076923

In [19]:
clf_lr.score(test_x_vectors, test_y)

0.8052884615384616

In [28]:
from sklearn.metrics import f1_score

In [29]:
print(f1_score(test_y, clf_svm.predict(test_x_vectors), average=None, labels = [Sentiment.NEGATIVE, Sentiment.NEUTRAL, Sentiment.POSITIVE]))
print(f1_score(test_y, clf_dec.predict(test_x_vectors), average=None, labels = [Sentiment.NEGATIVE, Sentiment.NEUTRAL, Sentiment.POSITIVE]))
print(f1_score(test_y, clf_gnb.predict(test_x_vectors.toarray()), average=None, labels = [Sentiment.NEGATIVE, Sentiment.NEUTRAL, Sentiment.POSITIVE]))
print(f1_score(test_y, clf_lr.predict(test_x_vectors), average=None, labels = [Sentiment.NEGATIVE, Sentiment.NEUTRAL, Sentiment.POSITIVE]))

[0.80952381 0.         0.80582524]
[0.65411765 0.         0.63882064]
[0.66508314 0.         0.65693431]
[0.80760095 0.         0.80291971]


./venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1464: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(
./venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1464: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(


In [ ]:
from sklearn.metrics import f1_score

In [21]:
test_set = ['Amazing book it was lit', 'horrible book do not buy', 'bad book do not buy']
new_test = vectorizer.transform(test_set)

clf_svm.predict(new_test)


array(['POSITIVE', 'NEGATIVE', 'NEGATIVE'], dtype='<U8')

In [35]:
f1_score(test_y, clf_svm.predict(test_x_vectors), average=None, labels = [Sentiment.NEGATIVE, Sentiment.NEUTRAL, Sentiment.POSITIVE])


array([0.80952381, 0.        , 0.80582524])

In [34]:
f1_score(test_y, clf_dec.predict(test_x_vectors), average=None, labels = [Sentiment.NEGATIVE, Sentiment.NEUTRAL, Sentiment.POSITIVE])


array([0.65411765, 0.        , 0.63882064])

In [33]:
f1_score(test_y, clf_svm.predict(test_x_vectors.toarray()), average=None, labels = [Sentiment.NEGATIVE, Sentiment.NEUTRAL, Sentiment.POSITIVE])


./venv/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1464: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true nor predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(


array([0.80952381, 0.        , 0.80582524])

In [32]:
f1_score(test_y, clf_svm.predict(test_x_vectors), average=None, labels = [Sentiment.NEGATIVE, Sentiment.NEUTRAL, Sentiment.POSITIVE])


array([0.80952381, 0.        , 0.80582524])

In [19]:
train_x[0]

"Olivia Hampton arrives at the Dunraven family home as cataloger of their extensive library. What she doesn't expect is a broken carriage wheel on the way. Nor a young girl whose mind is clearly gone, an old man in need of care himself (and doesn&#8217;t quite seem all there in Olivia&#8217;s opinion). Furthermore, Marion Dunraven, the only sane one of the bunch and the one Olivia is inexplicable drawn to, seems captive to everyone in the dusty old house. More importantly, she doesn't expect to fall in love with Dunraven's daughter Marion.Can Olivia truly believe the stories of sadness and death that surround the house, or are they all just local neighborhood rumor?Was that carriage trouble just a coincidence or a supernatural sign to stay away? If she remains, will the Castle&#8217;s dark shadows take Olivia down with them or will she and Marion long enough to declare their love?Patty G. Henderson has created an atmospheric and intriguing story in her Gothic tale. I found this to be a

In [82]:
# The following are checks on the data

train_y.count(Sentiment.POSITIVE)
test_y.count(Sentiment.POSITIVE)


2767

In [79]:
train_y.count(Sentiment.NEUTRAL)

0

In [86]:
train_y.count(Sentiment.NEGATIVE)
test_y.count(Sentiment.NEGATIVE)

208

In [22]:
# Tuning our model (with Grid Search)

In [42]:
from sklearn.model_selection import GridSearchCV

parameters = {'kernel': ('rbf', 'linear'), 'C': (1,4,8,16, 32)}

svc = svm.SVC()
clf = GridSearchCV(svc, parameters, cv=3)
clf.fit(train_x_vectors, train_y)


GridSearchCV(cv=3, estimator=SVC(),
             param_grid={'C': (1, 4, 8, 16, 32), 'kernel': ('rbf', 'linear')})